- trf
    - ca37: distance=passthrough
    - fa15: distance=pwr
- imp
    - 2474: tune=False
    - 20cf: tune=True
- clf
    - 2c44: ensemble=True, time=20
    - 25f2: ensemble=False, time=20

In [ ]:
from LiveAMP import *
T = Term(cycle_day=0, term_code=202308)
T.get_adm()

In [ ]:
df = pd.DataFrame()
df['a'] = [2,1,pd.NA]
df.sort_values(['a','a'])
df.query('a>1')

In [ ]:
variable = 'gender'
self.mlt.query(f"variable=='{variable}'").droplevel(['variable']).rename_axis(index={'value':variable})

In [1]:
from LiveAMP import *
# Flags().run()
self = AMP(cycle_day=119, styp_code='n', crse_code='_allcrse', param=cartesian(param_dcts)[0])
# self.get_Y()
self.get_X()
# variable = 'gender'
# self.get_result(variable)
# self.mlt.query(f"variable=='{variable}'").droplevel(['variable']).rename_axis(index={'value':variable})
# self.qry(self.mlt, variable).rename_axis(index={'value':variable})
# self.get_terms()
# self = run_amp(119, overwrite=[])

creating X.pkl: Elapsed time: 5.3332 seconds


KeyError: "['variable'] not in index"

In [6]:
self.mlt

current  admitted  final  \
pred_code levl_code styp_code variable value                             
202108    ug        n         _allcrse True      1104      1975   2142   
                              acct2301 True        10        38     40   
                              acct2302 True         0         0      1   
                              acom1110 True         4        16     16   
                              acom2307 True         0         1      1   
...                                               ...       ...    ...   
202308    ug        r         lgcy     True        41        70    112   
                    t         gender   f          297       476    740   
                                       m          153       249    469   
                              lgcy     False      331       538    867   
                                       True       119       187    342   

                                                   mlt  
pred_code levl_code styp_code variable value            
202108    ug        n         _allcrse True   1.084557  
                              acct2301 True   1.052632  
                              acct2302 True       <NA>  
                              acom1110 True        1.0  
                              acom2307 True        1.0  
...                                                ...  
202308    ug        r         lgcy     True        1.6  
                    t         gender   f      1.554622  
                                       m      1.883534  
                              lgcy     False  1.611524  
                                       True   1.828877  

[3385 rows x 4 columns]

In [5]:
self.X[1][[]].join(self.Y[0].droplevel(col)).query(qry).groupby(['pred_code','levl_code','styp_code','variable','value']).size()

pred_code  levl_code  styp_code  variable  value
202108     ug         n          _allcrse  True     1975
                                 acct2301  True       38
                                 acom1110  True       16
                                 acom2307  True        1
                                 aest1101  True        6
                                                    ... 
202308     ug         t          univ0342  True        1
                                 univ1100  True       17
                                 wses2322  True        1
                                 wses2405  True       11
                                 wses2451  True        2
Length: 2730, dtype: int64

In [19]:
self.X[1].melt(ignore_index=False)

variable value
id       pidm     pred_code levl_code styp_code                 
9175043  9176350  202208    ug        n          cycle_day   119
9175056  9175791  202208    <NA>      <NA>       cycle_day  <NA>
9175057  9175788  202208    gr        g          cycle_day   119
9175060  9175795  202208    ug        t          cycle_day   119
9175062  9175789  202208    ug        t          cycle_day   119
...                                                    ...   ...
<NA>     10485031 202108    <NA>      <NA>           final  12.0
         10485284 202208    <NA>      <NA>           final   6.0
         10485449 202308    <NA>      <NA>           final   9.0
                  202408    <NA>      <NA>           final  12.0
         10485716 202108    <NA>      <NA>           final   6.0

[11821722 rows x 2 columns]

In [16]:
col = ['levl_code','styp_code']
qry = f"levl_code == 'ug' & styp_code in ('n','r','t') & pred_code!={self.proj_code}"
variables = ['gender','lgcy']
aggy = lambda y: y.query(qry).groupby(['pred_code','levl_code','styp_code','variable','value']).size()
aggx = lambda x: aggy(x[variables].melt(ignore_index=False))

def get_df(dct):
    Y = pd.DataFrame(dct).fillna(0).prep()
    Y['mlt'] = Y['final'] / Y['admitted']
    Y[np.isinf(Y)] = pd.NA
    return Y

AY = get_df({
    'current': aggy(self.Y[1]),
    'admitted': aggy(self.X[1][[]].join(self.Y[0].droplevel(col))),
    'final': aggy(self.Y[0]),
})

AX = get_df({
    'current': aggx(self.X[1].query("current.notnull()")),
    'admitted': aggx(self.X[1].query("final.notnull()")),
    'final': aggx(self.X[0].query("final.notnull()")),
})

pd.concat([AX,AY])

current  admitted  final  \
pred_code levl_code styp_code variable value                             
202108    ug        n         gender   f          802      1321   1388   
                                       m          302       654    750   
                              lgcy     False      745      1329   1455   
                                       True       359       646    683   
                    r         gender   f           67       113    200   
...                                               ...       ...    ...   
202308    ug        t         univ0342 True         0         1      2   
                              univ1100 True         4        17     40   
                              wses2322 True         0         1      1   
                              wses2405 True         7        11     14   
                              wses2451 True         1         2      2   

                                                   mlt  
pred_code levl_code styp_code variable value            
202108    ug        n         gender   f      1.050719  
                                       m      1.146789  
                              lgcy     False  1.094808  
                                       True   1.057276  
                    r         gender   f      1.769912  
...                                                ...  
202308    ug        t         univ0342 True        2.0  
                              univ1100 True   2.352941  
                              wses2322 True        1.0  
                              wses2405 True   1.272727  
                              wses2451 True        1.0  

[3385 rows x 4 columns]

In [25]:
self.X[1].disp()

,,,,,cycle_day,apdc_day,appl_day,birth_day,end_date,cycle_date,apdc_date,appl_date,birth_date,term_code_entry,pred_desc,appl_no,levl_desc,styp_desc,admt_code,admt_desc,apst_code,apst_desc,apdc_code,apdc_desc,camp_code,camp_desc,majr_code,cnty_code,cnty_desc,stat_code,stat_desc,zip,natn_code,natn_desc,international,race_american_indian,race_asian,race_black,race_pacific,race_white,race_hispanic,gender,lgcy_code,lgcy_desc,resd_code,resd_desc,hs_pctl,flg_date,fafsa_app,finaid_accepted,disb_req_complete,schlship_app,math,reading,writing,gap_score,ssb,waiver,oriented,verified,act_equiv,distance,hs_qrtl,remote,resd,lgcy,majr_desc,dept_code,dept_desc,coll_code,coll_desc,cycle_day_missing,apdc_day_missing,appl_day_missing,birth_day_missing,end_date_missing,cycle_date_missing,apdc_date_missing,appl_date_missing,birth_date_missing,term_code_entry_missing,pred_code_missing,pred_desc_missing,pidm_missing,id_missing,appl_no_missing,levl_code_missing,levl_desc_missing,styp_code_missing,styp_desc_missing,admt_code_missing,admt_desc_missing,apst_code_missing,apst_desc_missing,apdc_code_missing,apdc_desc_missing,camp_code_missing,camp_desc_missing,majr_code_missing,cnty_code_missing,cnty_desc_missing,stat_code_missing,stat_desc_missing,zip_missing,natn_code_missing,natn_desc_missing,international_missing,race_american_indian_missing,race_asian_missing,race_black_missing,race_pacific_missing,race_white_missing,race_hispanic_missing,gender_missing,lgcy_code_missing,lgcy_desc_missing,resd_code_missing,resd_desc_missing,hs_pctl_missing,flg_date_missing,fafsa_app_missing,finaid_accepted_missing,disb_req_complete_missing,schlship_app_missing,math_missing,reading_missing,writing_missing,gap_score_missing,ssb_missing,waiver_missing,oriented_missing,verified_missing,act_equiv_missing,distance_missing,hs_qrtl_missing,remote_missing,resd_missing,lgcy_missing,majr_desc_missing,dept_code_missing,dept_desc_missing,coll_code_missing,coll_desc_missing,current,final
id,pidm,pred_code,levl_code,styp_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9175043,9176350,202208,ug,n,119,166,176,6717,2022-09-07,2022-05-11,2022-03-25,2022-03-15,2004-04-17,202208,fall 2022,1,undergraduate,new first time,fr,freshman (7th sem/graduated),d,decision made,r1,admitted (rp1),s,stephenville,psy,161,mclennan,tx,texas,76705,<NA>,<NA>,False,False,False,True,False,False,False,f,<NA>,<NA>,r,texas resident,84,2022-05-11,True,False,True,False,False,True,True,70,True,True,True,w,17.0,6987.96,1,False,True,False,psychology,psys,psychological sciences,ed,education,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,<NA>,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
final

pred_code  levl_code  styp_code  variable  value
202108     ug         n          gender    f        1321
                                           m         654
                                 lgcy      False    1329
                                           True      646
                      r          gender    f         113
                                           m          57
                                 lgcy      False      99
                                           True       71
                      t          gender    f         455
                                           m         235
                                 lgcy      False     513
                                           True      177
202208     ug         n          gender    f        1478
                                           m         810
                                 lgcy      False    1606
                                           True      682
                      r          gender

In [4]:
self.X[0]

,,,,,cycle_day,apdc_day,appl_day,birth_day,end_date,cycle_date,apdc_date,appl_date,birth_date,term_code_entry,...,hs_qrtl_missing,remote_missing,resd_missing,lgcy_missing,majr_desc_missing,dept_code_missing,dept_desc_missing,coll_code_missing,coll_desc_missing,credit_hr
id,pidm,pred_code,levl_code,styp_code,,,,,,,,,,,,,,,,,,,,,
9180867,9182174,202308,ug,n,0,265,265,7099,2023-09-13,2023-09-13,2022-12-22,2022-12-22,2004-04-06,202306,...,False,False,False,False,False,False,False,False,False,13.0
9176093,9176824,202208,ug,n,0,176,203,6616,2022-09-07,2022-09-07,2022-03-15,2022-02-16,2004-07-27,202208,...,True,False,False,False,False,False,False,False,False,16.0
9206862,9199911,202108,ug,t,0,165,169,7062,2021-09-08,2021-09-08,2021-03-27,2021-03-23,2002-05-09,202108,...,True,False,False,False,False,False,False,False,False,13.0
9210630,9210909,202308,ug,t,0,147,153,8700,2023-09-13,2023-09-13,2023-04-19,2023-04-13,1999-11-18,202308,...,True,False,False,False,False,False,False,False,False,6.0
9229821,9230040,202308,ug,n,0,286,287,6817,2023-09-13,2023-09-13,2022-12-01,2022-11-30,2005-01-13,202308,...,False,False,False,False,False,False,False,False,False,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9670713,9671953,202108,ug,n,0,211,287,7023,2021-09-08,2021-09-08,2021-02-09,2020-11-25,2002-06-17,202108,...,False,False,False,False,False,False,False,False,False,14.0
9193384,9193601,202208,ug,n,0,359,363,6764,2022-09-07,2022-09-07,2021-09-13,2021-09-09,2004-03-01,202208,...,False,False,False,False,False,False,False,False,False,15.0
9666758,9668063,202108,ug,n,0,280,294,6910,2021-09-08,2021-09-08,2020-12-02,2020-11-18,2002-10-08,202108,...,False,False,False,False,False,False,False,False,False,14.0


In [17]:
self.X[0].vc('value')

,ct,pct
value,,
True,11933,100.0


In [30]:
self.Y[1]

,,,,,,variable,credit_hr
pidm,pred_code,value,id,levl_code,styp_code,,
9687970,202108,True,9678539,ug,n,agri1100,1.0
9674251,202108,True,9674034,ug,n,agri1100,1.0
9667498,202108,True,9674445,ug,n,agri1100,1.0
9680112,202108,True,9679384,ug,n,agri1100,1.0
9690486,202108,True,9689241,ug,n,agri1100,1.0
...,...,...,...,...,...,...,...
9209673,202408,True,9208434,ug,n,_allcrse,6.0
9262708,202408,True,9262486,ug,t,_allcrse,6.0
9261014,202408,True,9259760,ug,n,_allcrse,6.0


In [39]:
self.Y[0]

,,,,,variable,credit_hr
pidm,pred_code,levl_code,styp_code,value,,
9687970,202108,ug,n,True,agri1100,1.0
9674251,202108,ug,n,True,agri1100,1.0
9667498,202108,ug,n,True,agri1100,1.0
9680112,202108,ug,n,True,agri1100,1.0
9690486,202108,ug,n,True,agri1100,1.0
...,...,...,...,...,...,...
9214006,202408,ug,n,True,_allcrse,6.0
10197455,202408,ug,r,True,_allcrse,6.0
9263456,202408,ug,t,True,_allcrse,9.0


In [41]:
attr = ['gender','lgcy']
grp = ['pred_code','levl_code','styp_code','variable','value']
# X = self.X[1][attr].melt(ignore_index=False)
aggy = lambda y: y.groupby(grp).size()
aggx = lambda x: aggy(x[attr].droplevel('value').melt(ignore_index=False))
current = aggx(self.X[1].query('value.notnull()'))
final = aggx(self.X[0])
# current = self.X[1].query("credit_hr>0")[attr].melt(ignore_index=False).groupby(grp).size()
X = self.X[0].droplevel('value').join(self.X[1][[]])
admitted = aggx(X.query("value.notnull()"))



current = aggy(self.Y[1])
final = aggy(self.Y[0])
Y = self.Y[0].droplevel('value').join(self.Y[1][[]])
admitted = aggy(Y.query("value.notnull()"))
final = aggy(Y)

# final = X.groupby(grp).size()

# current = self.Y[1].query("enrolled_ind").melt(ignore_index=False).groupby(grp).size()

# self.X[1].vc('enrolled_ind')

# current#.disp(4)
# Y
# admitted
final.query("variable=='_allcrse'")
# self.X[1]
# X

pred_code  levl_code  styp_code  variable  value
202108     ug         n          _allcrse  True     3861
                      r          _allcrse  True      184
                      t          _allcrse  True     1120
202208     ug         n          _allcrse  True     3898
                      r          _allcrse  True      161
                      t          _allcrse  True     1223
202308     ug         n          _allcrse  True     4036
                      r          _allcrse  True      230
                      t          _allcrse  True     1631
202408     ug         n          _allcrse  True     5335
                      r          _allcrse  True      482
                      t          _allcrse  True     2041
Name: 0, dtype: int64

In [46]:
self.Y[1].vc('value')

,ct,pct
value,,
True,25496,100.0


In [44]:
aggy(self.Y[0])

pred_code  levl_code  styp_code  variable  value
202108     ug         n          _allcrse  True     2142
                                 acct2301  True       40
                                 acct2302  True        1
                                 acom1110  True       16
                                 acom2307  True        1
                                                    ... 
202408     ug         t          univ0204  True        2
                                 univ0314  True        2
                                 univ1100  True        3
                                 wses2405  True        4
                                 wses3385  True        1
Length: 3921, dtype: int64

In [45]:
self.Y[0]

,,,,,variable,credit_hr
pidm,pred_code,levl_code,styp_code,value,,
9687970,202108,ug,n,True,agri1100,1.0
9674251,202108,ug,n,True,agri1100,1.0
9667498,202108,ug,n,True,agri1100,1.0
9680112,202108,ug,n,True,agri1100,1.0
9690486,202108,ug,n,True,agri1100,1.0
...,...,...,...,...,...,...
9214006,202408,ug,n,True,_allcrse,6.0
10197455,202408,ug,r,True,_allcrse,6.0
9263456,202408,ug,t,True,_allcrse,9.0


In [19]:
# self.X[1].melt(ignore_index=False).droplevel(['id','idx'])
grp = ['pred_code','levl_code','styp_code','variable','value']
col = ['gender','lgcy']
aggy = lambda y: y.groupby(grp).size()
aggx = lambda x: aggy(x[col].melt(ignore_index=False))
aggx(self.X[0])
aggx(self.X[0].join(self.X[1][[]].rsindex(['pidm','pred_code']), how='inner'))
# aggy(self.Y[1])
# self.X[0]
# self.X[0]['credit_hr'].min()
# self.X[1]
# self.X[0][[]].rsindex(['pidm','pred_code'])

pred_code  levl_code  styp_code  variable  value
202108     ug         n          gender    f        1320
                                           m         651
                                 lgcy      False    1326
                                           True      645
                      r          gender    f          67
                                           m          31
                                 lgcy      False      59
                                           True       39
                      t          gender    f         385
                                           m         191
                                 lgcy      False     432
                                           True      144
202208     ug         n          gender    f        1479
                                           m         810
                                 lgcy      False    1606
                                           True      683
                      r          gender

In [ ]:
a = self.y0.query(f"styp_code=='n' & variable=='_allcrse' & credit_hr>0").groupby(['pred_code','styp_code']).size()
# b = self.Y[0].query(f"variable=='_allcrse' & styp_code=='n'").groupby(['pred_code','styp_code']).sum()
print(a)
self.X0.query(f"styp_code=='n' & enrolled_ind").groupby(['pred_code','styp_code']).size()

In [ ]:
y.vc('styp_code')

In [ ]:
self.get_terms()
self.terms[0][202308].cycle_date

In [ ]:
# qry = f"select * from opeir.registration_fall2023 A where A.current_date={dt('2023-09-13')} and pidm=1108765"
qry = f"select * from opeir.registration_fall2021 A where pidm=1026466"
# 1111252"
db.execute(qry).disp(1000)

In [ ]:
y = self.y0.to_frame().query(f"variable=='_allcrse' & credit_hr>0 & pred_code<202408").reset_index(['levl_code','styp_code','variable'], drop=True)
# y = self.y0.to_frame().query(f"variable=='_allcrse'").reset_index('variable', drop=True).reset_index(['levl_code','styp_code'])
X = self.X0.query(f"styp_code=='n' & enrolled_ind")[['enrolled_ind']]
Z = X.join(y, how="left")
Z[Z.isnull().any(axis=1)]#.vc('pred_code')
# y.join(X).vc(['pred_code','styp_code'])

In [ ]:
y = self.y0.to_frame().query(f"variable=='_allcrse' & credit_hr>0 & levl_code=='ug' & styp_code=='n'").reset_index(['levl_code','styp_code','variable'], drop=True)
# y = self.y0.to_frame().query(f"variable=='_allcrse'").reset_index('variable', drop=True).reset_index(['levl_code','styp_code'])
X = self.X0#.query(f"styp_code=='n' & enrolled_ind")[[]]
# Z = X.join(y, how="left")
# Z[Z.isnull().any(axis=1)]
y.join(X).vc(['pred_code','styp_code'])

In [ ]:
self.get_X()
self.mlt

In [ ]:
self.Y

In [ ]:
self.Y

In [ ]:
qry = f"""
select
    A.*,
    (select B.stvlgcy_desc from stvlgcy B where B.stvlgcy_code = A.lgcy_code) as lgcy_desc
from (
    select
        A.*,
        (select B.spbpers_lgcy_code from spbpers B where B.spbpers_pidm = A.pidm) as lgcy_code
    from
        opeir.admissions_fall2023 A
    where enrolled_ind='Y' and A.current_date = {dt('2023-09-13')}
) A
--fetch first 1 rows only
"""
X = db.execute(qry)

In [ ]:
X.vc(['lgcy_code','lgcy_desc'])
# X['lgcy_code'].isin([pd.NA,'o']).sum()

In [ ]:
self.mlt

In [ ]:
self.y['final']
# self.credit

In [ ]:
self.X.vc(['pred_code','lgcy'])

In [ ]:
self.credit
g = lambda y: y.query(f"variable=='{self.crse_code}'").droplevel('variable')
y = self.y['current']
y.squeeze().rename_axis(index={'variable':'crse_code'}).rename('d')
# g(self.y['current'])

In [ ]:
# for k,v in self.y.items():
#     print(k)
#     v.disp(4)
self.mlt.query("styp_code=='n'").head(10)
# self.__dict__.keys()
# self.credit

In [ ]:
self.X.apdc_code

In [ ]:
attr = ['gender','lgcy']
Z = self.X_final[attr].melt(ignore_index=False).astype('string')
A1 = Z.groupby(['styp_code','pred_code','variable','value']).size()

In [ ]:
ren = {'term_code':'pred_code', 'term_desc':'pred_desc', 'index':'idx', 'crse_code':'variable'}
Y = [pd.concat([
        T.reg
        .rename(columns=ren)
        .set_index(['pidm','levl_code','styp_code','pred_code','variable'])
        ['credit_hr']
        .fillna(0)
        .query('credit_hr>0')
        .rename('value')
        .to_frame()
        > 0
        
    for pred_code, T in dct.items()]) for dct in self.terms]
Y[1].rename(columns={'a':'b'})
# A2 = Y[1].groupby(['styp_code','pred_code','variable','value']).size()

In [ ]:
pd.concat([A1,A2])

In [ ]:
dct = self.terms[1]
ren = {'term_code':'pred_code', 'term_desc':'pred_desc', 'index':'idx', 'crse_code':'variable'}
R = pd.concat([T.raw.rename(columns=ren) for pred_code, T in dct.items()]).copy().dropna(axis=1, how='all').reset_index(drop=True).prep()
repl = {'ae':0, 'n1':1, 'n2':2, 'n3':3, 'n4':4, 'r1':1, 'r2':2, 'r3':3, 'r4':4}
R['hs_qrtl'] = pd.cut(R['hs_pctl'], bins=[-1,25,50,75,90,101], labels=[4,3,2,1,0], right=False).combine_first(R['apdc_code'].map(repl))
R['remote'] = R['camp_code'] != 's'
R['resd'] = R['resd_code'] == 'r'
R['oriented'] = R['oriented'] != 'n'
R['lgcy'] = ~R['lgcy_code'].isin(['n','o'])
R['majr_code'] = R['majr_code'].replace({'0000':pd.NA, 'und':pd.NA, 'eled':'eted', 'agri':'unda'})
R['coll_code'] = R['coll_code'].replace({'ae':'an', 'eh':'ed', 'hs':'hl', 'st':'sm', '00':pd.NA})
R['coll_desc'] = R['coll_code'].map({
    'an': 'ag & natural_resources',
    'ba': 'business',
    'ed': 'education',
    'en': 'engineering',
    'hl': 'health sciences',
    'la': 'liberal & fine arts',
    'sm': 'science & mathematics',
    pd.NA: 'no college designated',
})
majr = ['majr_desc','dept_code','dept_desc','coll_code','coll_desc']
S = R.sort_values('cycle_date').drop_duplicates(subset='majr_code', keep='last')[['majr_code',*majr]]
X = R.drop(columns=majr).merge(S, on='majr_code', how='left').prep(bool=True)
checks = [
    'cycle_day >= 0',
    'apdc_day >= cycle_day',
    'appl_day >= apdc_day',
    'birth_day >= appl_day',
    'birth_day >= 5000',
    'distance >= 0',
    'hs_pctl >=0',
    'hs_pctl <= 100',
    'hs_qrtl >= 0',
    'hs_qrtl <= 4',
    'act_equiv >= 1',
    'act_equiv <= 36',
    'gap_score >= 0',
    'gap_score <= 100',
]
for check in checks:
    mask = X.eval(check)
    assert mask.all(), [check,X[~mask].disp(5)]

fill = {
    'birth_day': ['median',['pred_code','styp_code']],
    'remote': False,
    'international': False,
    **{f'race_{r}': False for r in ['american_indian','asian','black','pacific','white','hispanic']},
    'lgcy': False,
    'resd': False,
    'waiver': False,
    'fafsa_app': False,
    'schlship_app': False,
    'finaid_accepted': False,
    'ssb': False,
    'math': False,
    'reading': False,
    'writing': False,
    'gap_score': 0,
    'oriented': False,
    # 'coll_code': 'na',
    # 'coll_desc': 'na',
}
for k, v in fill.items():
    X[k] = X.impute(k, *listify(v))
M = X.isnull().rename(columns=lambda x:x+'_missing')
attr = ['idx','id','pidm','levl_code','styp_code','pred_code']
X = X.join(M).sample(frac=1, random_state=self.param['imp'][2]['random_state']).reset_index().rename(columns=ren).prep(bool=True).set_index(attr).sort_index()
X.disp(4)

In [ ]:
X

In [ ]:
y = pd.concat([
        T.reg
        .rename(columns=ren)
        .set_index(['pidm','pred_code','variable'])
        # .set_index(['pidm','pred_code','crse_code'])
        [['credit_hr','levl_code','styp_code']]
        # .query('credit_hr>0')
        # .rename('value')
    for pred_code, T in dct.items()])#.reset_index()#.unstack().fillna(0)#.prep()
y

In [ ]:
y

In [ ]:
cols = ['levl_code_x','levl_code_y','styp_code_x','styp_code_y']
Z = y.reset_index().merge(X[[]].reset_index(), on=['pidm','pred_code'], how='outer').fillna({k:'' for k in cols})
# Z = Z[['levl_code_x','levl_code_y','styp_code_x','styp_code_y']].fillna('')
msk = (Z['levl_code_x'] != Z['levl_code_y']) | (Z['styp_code_x'] != Z['styp_code_y'])
Z[msk].vc(cols).sort_index().reset_index()
# Z

In [ ]:
Z = X[[]].join(y, how='right').reset_index()

In [ ]:
y.reset_index()

In [ ]:
y.query("variable=='_allcrse'").drop(columns='variable')

In [ ]:
r = y.query("variable=='_allcrse' & value>0").droplevel('variable').rename('registered')>0
r

In [ ]:
y.to_frame().join(r)

In [ ]:
attr = ['gender','lgcy']
Z = X[attr].melt(ignore_index=False).astype('string').join(r)
Z

In [ ]:
y.join(y.query("variable=='_allcrse'").droplevel('variable').rename('registered')>0)

In [ ]:
Z.groupby(['styp_code','pred_code','variable','value'])['registered'].sum()

In [ ]:
Z = X.join(y).melt(ignore_index=False).astype('string')

In [ ]:
Z.dtypes

In [ ]:
Z.prep().dtypes

In [ ]:
pd.concat([X.melt(ignore_index=False).reset_index(), y])

In [ ]:
self.X

In [ ]:
self.get_X()
self.mlt.query("crse_code=='_allcrse'")

In [ ]:
A = self.summary.groupby('crse_code')['pred_code_actual'].agg(['min','max'])
A.query('min>1 & min<5').sort_values('max', ascending=False)

In [ ]:
import numpy as np
x = np.arange(100)
100 * (1 - (x%5==0).cumsum() / (x+1))

In [ ]:
self.train_score
self.get_X()
nm='crse_code'
grp = uniquify([nm,'crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
Z = self.X.join(self.Y, how='inner').reset_index()
S = Z.groupby(grp).apply(lambda y: pd.Series({
    'pred_code_actual': y['actual'].sum(),
    'pred_code_predicted': y['proba'].sum(),
    'test_score': log_loss(y['actual'], y['proba'], labels=[False,True]) * 100,
}), include_groups=False).join(self.train_score)


In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt3/119/rslt_crse_code/n/_allcrse/fa15/77d7/d6c4.pkl')
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt5/119/rslt_crse_code/n/_allcrse/fa15/77d7/bf3e.pkl')
print(w['param']['clf'][2]['time_budget'])
w['rslt_crse_code'][' 50%']

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt4/119/rslt_crse_code/n/_allcrse/fa15/77d7/d6c4.pkl')
w['rslt_crse_code'][' 50%']

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt5/119/rslt_crse_code/n/_allcrse/fa15/77d7/b66c.pkl')
w['rslt_crse_code'][' 50%']

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/119/rslt_crse_code/n/_allcrse/fa15/77d7/114b.pkl')
w['rslt_crse_code'][' 50%']

In [ ]:
y1 = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/119/Y/n/_allcrse/fa15/77d7/b66c.pkl')['Y']
y2 = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/119/Y/n/_allcrse/fa15/77d7/b66c.pkl')['Y']
y2b = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/119/Y/n/_allcrse/fa15/77d7/114b.pkl')['Y']
y3 = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt3/119/Y/n/_allcrse/fa15/77d7/b66c.pkl')['Y']
nm = 'crse_code'
def g(y):
    grp = uniquify([nm,'crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
    z = y.groupby(grp)[['actual','proba']].sum()
    grp.remove('sim')
    return z.groupby(grp).mean()

In [ ]:
g(y1)

In [ ]:
g(y2)

In [ ]:
g(y2b)

In [ ]:
g(y3)

In [ ]:
g(y3) - g(y2)